# Lösningsförslag till Övningsuppgift - Stopped Flow
Författare: Björn Dahlgren, 2016-04-04

Se övningsuppgiften för detaljer. Kortfattat ska vi bestämma hastighetskonstanten för följande reaktion:

$$
3A + B \rightarrow C
$$

Detta lösningsförslag är skrivet i Python, se bilaga laborationshandledning för liknande exempel skrivna för Matlab.

In [ ]:
from __future__ import division, print_function
import numpy as np
from numpy.polynomial import polynomial as P
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
%matplotlib inline

Från uppgiftstexten vet vi att vi har data för 5 värden på [B]0, 10 replikat för vardera, och 42 tidpunkter per mätserie. Vi läser in filerna in i variabeln ``data``:

In [ ]:
n_conc, n_rep, n_t = 5, 10, 42
conc_B = np.linspace(1/10, n_conc/10, n_conc)  # molar
data = np.empty((n_conc, n_rep, n_t, 2))
for i in range(n_conc):
    for j in range(n_rep):
        data[i, j, :, :] = np.loadtxt('data/0.%d/%d.dat' % (i+1, j+1))

Om vi antar pseudo tredje ordningens reaktion ($[B]_0 \gg [A]_0$) blir hastighetsuttrycket för reaktionen:

$$
\frac{dA}{dt} = -3kA^3
$$

integrering ger:
$$
A(t) = \frac{1}{\sqrt{\frac{1}{A_0^2} + 6kt}}
$$

Detta uttryck skapar vi en funktion för som vi kallar "``model``":

In [ ]:
def model(t, y0, k):
    return (y0**-2 + 6*k*t)**-0.5

Nu genomför vi ickelinjärkurvanpassning av vår modellekvation på var och en av dataserierna. Vi formulerar även en relativ vikt baserad på hur väl passningen är. Valet av vikt är något godtyckligt och det kan vara bra att prova sig fram, ett vanligt val är Mean Square Deviation (MSD) & Mean Absolute Deviation (MAD). Vi använder MSD nedan:

In [ ]:
popt = np.empty((n_conc, n_rep, 2))  # y0, k
pcov = np.empty((n_conc, n_rep, 2, 2))
all_k = np.empty((n_conc, n_rep))
all_w = np.empty((n_conc, n_rep))
plt.figure(figsize=(16, 4))
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.set_yscale('log')
    for j in range(n_rep):
        t = data[i, j, :, 0]
        y = data[i, j, :, 1]
        popt[i, j, :], pcov[i, j, :, :] = curve_fit(model, t, y, [y[0], 1])
        m = model(t, *popt[i, j, :])
        plt.plot(t, y, 'k', alpha=0.4)
        plt.plot(t, m, 'b')
        all_k[i, j] = popt[i, j, 1]
        #all_w[i, j] = 1/pcov[i, j, 1, 1]             # varians i k
        #all_w[i, j] = 1/pcov[i, j, 0, 0]             # varians i y0
        all_w[i, j] = 1/np.average(np.square(y-m))    # MSD
        #all_w[i, j] = 1/(np.average(np.abs(y-m)))    # MAD

Nu beräknar vi de viktade medelvärdena (se laborationshandledning för formel) för de olika värdena för $[B]_0$ baserat på respektive 10 replikat.

In [ ]:
k = np.empty(n_conc)
for i in range(n_conc):
    accum_k, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_k += all_k[i, j]*w
        accum_w += w
    k[i] = accum_k/accum_w

Nu beräknar vi respektive viktade stickprovsvarians (se laborationshandledning för formel) för de nyligen beräknade viktade medelvärdena.

In [ ]:
dk = np.empty_like(k)
for i in range(n_conc):
    accum_D, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_D += w*(all_k[i, j] - k[i])**2
        accum_w += w
    dk[i] = accum_D/((n_rep - 1)*accum_w)

Våra pseudo tredje ordningens hastighetskonstanter är linjärt beroende av $[B]_0$:

$$
k_f = [B]_0 k
$$

Därför beräknar vi $k_f$ genom viktad linjär regression:

In [ ]:
plt.errorbar(conc_B, k, 3*dk**0.5, c='k', ls='None', marker='.')
p, cov = P.polyfit(conc_B, k, 1, w=1/dk, full=True)
plt.plot(conc_B, P.polyval(conc_B, p), 'b', label='$k_f$ = %.2f' % p[1])
plt.gca().set_xlim((0, conc_B[-1]*1.05))
plt.gca().set_ylim((0, k[-1]*1.1))
plt.legend(loc='best')
print(p)

Och därmed är uppgiften löst. Värdet på $k_f$ känns bekannt (eftersom våra indata är syntetiska stog det mig fritt att välja det "sanna" $k_f$).

Som "överkurs" kan vi ta fram ett antal statistiska värden med hjälp av paketet "statsmodels":

In [ ]:
import statsmodels.api as sm
sm.WLS(k, sm.add_constant(conc_B), weights=1/dk).fit().summary()